In [20]:
# Builds the eval dataset

In [21]:
from pymilvus import MilvusClient, Collection
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex

import random as rnd

In [22]:
vector_store = MilvusVectorStore(
    uri="/Users/Zhiyuan/Project/rag_projects/my-web-novel/cache/db/milvus.db", collection_name='create_eval_dataset_collections_hybrid'
)
client = vector_store.client

In [23]:
# Embed model
embed_model = HuggingFaceEmbedding(model_name='/Users/Zhiyuan/Project/hf/bge-m3', embed_batch_size=1024)
Settings.embed_model = embed_model

In [24]:
import hashlib
print(hashlib.sha1('凡人修仙传.txt'.encode('utf-8')).hexdigest()[:8])
print(hashlib.sha1('凡人修仙之仙界篇.txt'.encode('utf-8')).hexdigest()[:8])

dfa3a46c
06ad4aec


In [34]:
client.list_collections()

In [30]:
client.describe_collection('create_eval_dataset_collections_hybrid')

{'collection_name': 'create_eval_dataset_collections_hybrid',
 'auto_id': False,
 'num_shards': 0,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 65535},
   'is_primary': True},
  {'field_id': 101,
   'name': 'embedding',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 1024}},
  {'field_id': 102,
   'name': 'sparse_embedding',
   'description': '',
   'type': <DataType.SPARSE_FLOAT_VECTOR: 104>,
   'params': {}}],
 'functions': [],
 'aliases': [],
 'collection_id': 0,
 'consistency_level': 0,
 'properties': {},
 'num_partitions': 0,
 'enable_dynamic_field': True}

In [16]:
# dfa3a46c00000000 -> dfa3a46c00011095
# 06ad4aec00000000 -> 06ad4aec00006915

In [9]:
def messages_builder_example(content):
    messages = [{"role": "system", "content": "你是一个武侠传记作家."}, {"role": "user", "content": content}]
    return messages

In [43]:
import json
import csv
import copy
import uuid
format_basic = {'idx': "", 'question':"", 'answer':"", "nodes":[], "context":[]}
prefix = 'dfa3a46c'
with open('eval_dataset_text_basic.jsonl', 'w', encoding='utf-8') as fw:
    with open('eval_dataset_text_basic.qwen.jsonl', 'w', encoding='utf-8') as fout:
        prefix = 'dfa3a46c'
        for _ in range(100):
            start_idx = rnd.randint(0, 11095)
            id_ = prefix+str(start_idx).zfill(8)
            res = client.query(
                collection_name="create_eval_dataset_collections",
                filter=f"id like \'{id_[:-2]}%\'"
            )
            ss = rnd.randint(1,45)
            context_pool = [res[i] for i in range(ss, ss+rnd.randint(1,50))]
            row_ = copy.deepcopy(format_basic)
            row_['nodes'] = [e['id'] for e in context_pool]
            row_['context'] = [json.loads(e['_node_content'])['text'] for e in context_pool]
            row_['idx'] = str(uuid.uuid4())
            fw.write(json.dumps(row_, ensure_ascii=False)+'\n')

            body = {"model": "qwen-plus", "messages": messages_builder_example((
                "根据以下提供的内容:\n-----\n"+
                "\n".join(row_["context"])+
                "\n-----\n提出一个根据上面的内容可以作答的问题，并给出这个问题的答案。问题要清晰具体，答案要简洁明了。总字数限制在200个字以内。"
            ))}
            request = {"custom_id": row_['idx'], "method": "POST", "url": "/v1/chat/completions", "body": body}
            fout.write(json.dumps(request, separators=(',', ':'), ensure_ascii=False) + "\n", )

        prefix = '06ad4aec'
        for _ in range(70):
            start_idx = rnd.randint(0, 6900)
            id_ = prefix+str(start_idx).zfill(8)
            res = client.query(
                collection_name="create_eval_dataset_collections",
                filter=f"id like \'{id_[:-2]}%\'"
            )
            ss = rnd.randint(1,50)
            context_pool = [res[i] for i in range(ss, ss+rnd.randint(1,50))]
            row_ = copy.deepcopy(format_basic)
            row_['nodes'] = [e['id'] for e in context_pool]
            row_['context'] = [json.loads(e['_node_content'])['text'] for e in context_pool]
            row_['idx'] = str(uuid.uuid4())
            fw.write(json.dumps(row_, ensure_ascii=False)+'\n')

            body = {"model": "qwen-plus", "messages": messages_builder_example((
                "根据以下提供的内容:\n-----\n"+
                "\n".join(row_["context"])+
                "\n-----\n提出一个根据上面的内容可以作答的问题，并给出这个问题的答案。总字数限制在200个字以内。"
            ))}
            request = {"custom_id": row_['idx'], "method": "POST", "url": "/v1/chat/completions", "body": body}
            fout.write(json.dumps(request, separators=(',', ':'), ensure_ascii=False) + "\n", )

In [44]:
import os
from pathlib import Path
from openai import OpenAI

dashscope_client = OpenAI(
    # 若没有配置环境变量，可用百炼API Key将下行替换为：api_key="sk-xxx"。但不建议在生产环境中直接将API Key硬编码到代码中，以减少API Key泄露风险。
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写百炼服务的base_url
)

# test.jsonl 是一个本地示例文件，purpose必须是batch
file_object = dashscope_client.files.create(file=Path('eval_dataset_text_basic.qwen.jsonl'), purpose="batch")
file_object

FileObject(id='file-batch-ocBVvmfiJy7FP9Et9HwHPoMq', bytes=5725463, created_at=1735367490, filename='eval_dataset_text_basic.qwen.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [45]:
batch = dashscope_client.batches.create(
    input_file_id=file_object.id,  # 上传文件返回的 id
    endpoint="/v1/chat/completions",  # 大语言模型固定填写，/v1/chat/completions
    completion_window="24h"  # 当前只支持24h，24小时未运行完会超时
)
print(batch)

Batch(id='batch_91a58757-b5e0-43ce-8593-571a6ce96f40', completion_window='24h', created_at=1735367491, endpoint='/v1/chat/completions', input_file_id='file-batch-ocBVvmfiJy7FP9Et9HwHPoMq', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=None, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [48]:
import time
while True:
    try:
        batch = dashscope_client.batches.retrieve(batch.id)  # 将batch_id替换为Batch任务的id
        content = dashscope_client.files.content(file_id=batch.output_file_id)
        # 打印结果文件内容
        # print(content.text)
        # 保存结果文件至本地
        content.write_to_file("result.jsonl")
        break
    except Exception as e:
        time.sleep(10)

In [67]:
dataset_ = []
for ii in range(3):
    with open(f'ss/eval_dataset_text_basic.{ii}.jsonl', 'r') as fr:
        dataset = [json.loads(e) for e in fr.readlines()]
    with open(f"ss/result.{ii}.jsonl", 'r') as fr:
        qas = {json.loads(e)['custom_id']: json.loads(e) for e in fr.readlines()}
    for row in dataset:
        row_ = copy.deepcopy(row)
        idx = row['idx']
        if idx not in qas:
            continue
        text = qas[idx]['response']['body']['choices'][0]['message']['content']
        assert '答案：' in text, print(text)
        assert '问题：' in text, print(text)
        row_['question'] = text.split('答案：')[0].split('问题：')[1].replace('\n', '').replace('*', '')
        row_['answer'] = text.split('答案：')[-1].replace('\n', '').replace('*', '')
        dataset_.append(json.dumps(row_, ensure_ascii=False))

In [68]:
with open('../data/eval/eval_dataset_text_basic.jsonl', 'w') as fh:
    print(f'{len(dataset_)} rows being recorded.')
    fh.write('\n'.join(dataset_))

150 rows being recorded.


In [ ]:
# prepare advanced dataset

In [73]:
dataset_adv = {}
with open('../data/eval/eval_dataset_text_adv.txt', 'r') as fh:
    with open('../data/eval/eval_dataset_text_adv.jsonl', 'w') as fw:
        raw_txt = fh.read()
        qas = raw_txt.split('\n\n\n\n')
        for qa in qas:
            try:
                q_, a_ = qa.split('a:')
                fw.write(json.dumps({'question':q_, 'idx':str(uuid.uuid4()), 'answer':a_, "nodes":[], "context":[]}, ensure_ascii=False)+'\n')
            except Exception as e:
                print(qa)
                raise e
            